## SETUP

In [1]:
import networkx as nx
import pandas as pd

In [2]:
df = pd.read_csv('edges.csv')
station_info = pd.read_csv('MTA_Subway_Stations_updated.csv')

In [3]:
display(df)

,Year,Month,Day of Week,Hour of Day,Timestamp,Origin Station Complex ID,Origin Station Complex Name,Origin Latitude,Origin Longitude,Destination Station Complex ID,Destination Station Complex Name,Destination Latitude,Destination Longitude,Estimated Average Ridership,Origin Point,Destination Point
0,2024,1,Monday,1,01/08/2024 01:00:00 AM,26,"DeKalb Av (B,Q,R)",40.690635,-73.981824,355,"Winthrop St (2,5)",40.656652,-73.950200,0.5556,POINT (-73.981824 40.690635),POINT (-73.9502 40.656652)
1,2024,1,Monday,1,01/08/2024 01:00:00 AM,231,"Grand St (B,D)",40.718267,-73.993753,284,Nassau Av (G),40.724635,-73.951277,0.3068,POINT (-73.993753 40.718267),POINT (-73.951277 40.724635)
2,2024,1,Monday,1,01/08/2024 01:00:00 AM,313,"72 St (1,2,3)",40.778453,-73.981970,71,8 Av (N),40.635064,-74.011719,0.3012,POINT (-73.98197 40.778453),POINT (-74.011719 40.635064)
3,2024,1,Monday,1,01/08/2024 01:00:00 AM,320,23 St (1),40.744081,-73.995657,309,103 St (1),40.799446,-73.968379,0.9000,POINT (-73.995657 40.744081),POINT (-73.968379 40.799446)
4,2024,1,Monday,1,01/08/2024 01:00:00 AM,399,68 St-Hunter College (6),40.768141,-73.963870,618,"14 St (A,C,E)/8 Av (L)",40.740335,-74.002134,0.2940,POINT (-73.96387 40.768141),POINT (-74.002134 40.740335)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9169120,2024,1,Sunday,12,01/14/2024 12:00:00 PM,426,"E 180 St (2,5)",40.841894,-73.873488,379,Mosholu Pkwy (4),40.879750,-73.884655,0.3322,POINT (-73.873488 40.841894),POINT (-73.884655 40.87975)
9169121,2024,1,Sunday,12,01/14/2024 12:00:00 PM,427,"West Farms Sq-E Tremont Av (2,5)",40.840295,-73.880049,613,"Lexington Av (N,R,W)/59 St (4,5,6)",40.762593,-73.967613,0.7305,POINT (-73.880049 40.840295),POINT (-73.967613 40.762593)
9169122,2024,1,Sunday,12,01/14/2024 12:00:00 PM,32,"36 St (D,N,R)",40.655144,-74.003549,263,"63 Dr-Rego Park (M,R)",40.729846,-73.861604,0.3130,POINT (-74.003549 40.655144),POINT (-73.861604 40.729846)
9169123,2024,1,Sunday,12,01/14/2024 12:00:00 PM,324,Houston St (1),40.728251,-74.005367,407,Astor Pl (6),40.730054,-73.991070,0.8602,POINT (-74.005367 40.728251),POINT (-73.99107 40.730054)


In [4]:
station_info = station_info[['Complex ID', 'Borough']]
display(station_info)

,Complex ID,Borough
0,1,Q
1,2,Q
2,3,Q
3,4,Q
4,5,Q
...,...,...
491,517,SI
492,518,SI
493,519,SI
494,522,SI


In [5]:
df2 = df

df2 = pd.merge(left=df2, right=station_info, left_on='Origin Station Complex ID', right_on='Complex ID', how='left')
df2 = pd.merge(left=df2, right=station_info, left_on='Destination Station Complex ID', right_on='Complex ID', how='left')
df2 = df2.rename(columns={'Borough_x': 'Origin Borough', 'Borough_y': 'Destination Borough'})
df2 = df2[['Origin Station Complex Name', 'Destination Station Complex Name', 'Day of Week', 'Hour of Day', 'Estimated Average Ridership', 'Origin Borough', 'Destination Borough']]
display(df2)

,Origin Station Complex Name,Destination Station Complex Name,Day of Week,Hour of Day,Estimated Average Ridership,Origin Borough,Destination Borough
0,"DeKalb Av (B,Q,R)","Winthrop St (2,5)",Monday,1,0.5556,Bk,Bk
1,"Grand St (B,D)",Nassau Av (G),Monday,1,0.3068,M,Bk
2,"72 St (1,2,3)",8 Av (N),Monday,1,0.3012,M,Bk
3,23 St (1),103 St (1),Monday,1,0.9000,M,M
4,68 St-Hunter College (6),"14 St (A,C,E)/8 Av (L)",Monday,1,0.2940,M,M
...,...,...,...,...,...,...,...
13922070,"36 St (D,N,R)","63 Dr-Rego Park (M,R)",Sunday,12,0.3130,Bk,Q
13922071,Houston St (1),Astor Pl (6),Sunday,12,0.8602,M,M
13922072,"Court Sq (E,G,M,7)",190 St (A),Sunday,12,0.9110,Q,M
13922073,"Court Sq (E,G,M,7)",190 St (A),Sunday,12,0.9110,Q,M


In [6]:
edges = (df2.groupby(['Origin Station Complex Name', 'Destination Station Complex Name', 'Day of Week', 'Hour of Day', 'Origin Borough', 'Destination Borough'])).mean().sort_values(by='Estimated Average Ridership', ascending=False).reset_index()
edges['edge_key'] = edges['Day of Week'] + ':' + edges['Hour of Day'].astype(str)
edges

,Origin Station Complex Name,Destination Station Complex Name,Day of Week,Hour of Day,Origin Borough,Destination Borough,Estimated Average Ridership,edge_key
0,"Grand Central-42 St (S,4,5,6,7)","Fulton St (A,C,J,Z,2,3,4,5)",Wednesday,8,M,M,1534.9962,Wednesday:8
1,"Grand Central-42 St (S,4,5,6,7)","Fulton St (A,C,J,Z,2,3,4,5)",Thursday,8,M,M,1456.8167,Thursday:8
2,"Grand Central-42 St (S,4,5,6,7)","Fulton St (A,C,J,Z,2,3,4,5)",Tuesday,8,M,M,1419.9016,Tuesday:8
3,"Fulton St (A,C,J,Z,2,3,4,5)","Grand Central-42 St (S,4,5,6,7)",Wednesday,17,M,M,1192.9002,Wednesday:17
4,"Fulton St (A,C,J,Z,2,3,4,5)","Grand Central-42 St (S,4,5,6,7)",Thursday,17,M,M,1097.7822,Thursday:17
...,...,...,...,...,...,...,...,...
9169120,Norwood-205 St (D),"Forest Hills-71 Av (E,F,M,R)",Saturday,13,Bx,Q,0.1250,Saturday:13
9169121,"Fulton St (A,C,J,Z,2,3,4,5)","Myrtle Av (M,J,Z)",Sunday,5,M,Bk,0.1250,Sunday:5
9169122,"167 St (B,D)","49 St (N,R,W)",Monday,0,Bx,M,0.1000,Monday:0
9169123,"72 St (C,B)","Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)",Monday,5,M,M,0.1000,Monday:5


In [7]:
print(edges[edges['Origin Borough'] == 'Q'])

        Origin Station Complex Name  \
141      Vernon Blvd-Jackson Av (7)   
143      Vernon Blvd-Jackson Av (7)   
155      Vernon Blvd-Jackson Av (7)   
271            Flushing-Main St (7)   
307            Flushing-Main St (7)   
...                             ...   
9169110  Vernon Blvd-Jackson Av (7)   
9169116                 36 St (M,R)   
9169117          46 St-Bliss St (7)   
9169118          61 St-Woodside (7)   
9169119          61 St-Woodside (7)   

                         Destination Station Complex Name Day of Week  \
141                       Grand Central-42 St (S,4,5,6,7)   Wednesday   
143                       Grand Central-42 St (S,4,5,6,7)    Thursday   
155                       Grand Central-42 St (S,4,5,6,7)     Tuesday   
271      Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)    Thursday   
307                       Grand Central-42 St (S,4,5,6,7)     Tuesday   
...                                                   ...         ...   
9169110             

In [8]:
G = nx.from_pandas_edgelist(edges, source="Origin Station Complex Name", target="Destination Station Complex Name", edge_attr=True, create_using=nx.MultiDiGraph, edge_key='edge_key')

In [9]:
named_station_info = pd.merge(left=df, right=station_info, left_on='Origin Station Complex ID', right_on='Complex ID', how='right')
named_station_info = named_station_info[['Borough', 'Complex ID', 'Origin Station Complex Name']]
named_station_info = named_station_info.groupby(['Origin Station Complex Name', 'Borough', 'Complex ID'], as_index=False).count().sort_values(by='Complex ID')

In [10]:
named_station_info.columns

Index(['Origin Station Complex Name', 'Borough', 'Complex ID'], dtype='object')

In [11]:

boroughs = {}
ids = list(named_station_info["Origin Station Complex Name"])
b = list(named_station_info["Borough"])
for i in range(len(named_station_info)):
    boroughs[ids[i]] = b[i]
nx.set_node_attributes(G, boroughs, name="Borough")

## 1

### 1A

In [12]:
## OUT DEGREE FOR ORIGIN

boroughs = ['M', 'Q', 'Bk', 'Bx', 'SI']
borough_ridership = {}
for b in boroughs:
    filtered = nx.subgraph_view(G, filter_node=lambda x: G.nodes[x]["Borough"] == b)
    ridership = []
    for n in filtered.nodes:
        ridership.append([filtered.out_degree(n, weight='Estimated Average Ridership'), n])
    ridership = sorted(ridership, reverse=True)
    borough_ridership[b] = ridership[:5]
print(borough_ridership)


{'M': [[492022.8909999953, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'], [375744.0561000017, 'Grand Central-42 St (S,4,5,6,7)'], [223684.4673000008, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'], [219993.42779999992, '34 St-Penn Station (A,C,E)'], [212053.69609999796, '34 St-Penn Station (1,2,3)']], 'Q': [[115028.43759999999, 'Flushing-Main St (7)'], [106116.53660000018, '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)'], [49340.88569999989, '103 St-Corona Plaza (7)'], [47856.67200000007, 'Junction Blvd (7)'], [42665.57040000028, 'Sutphin Blvd-Archer Av-JFK Airport (E,J,Z)']], 'Bk': [[101892.60110000017, 'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)'], [55409.06490000012, 'Jay St-MetroTech (A,C,F,R)'], [44169.229199999936, 'Court St (R)/Borough Hall (2,3,4,5)'], [39286.06430000026, 'Kings Hwy (B,Q)'], [38693.42919999942, 'Crown Hts-Utica Av (3,4)']], 'Bx': [[20482.494200000005, '161 St-Yankee Stadium (B,D,4)'], [18945.42069999996, '3 Av-149 St (2,5)'], [13748.588000000176, 'Hunts Poin

### 1B

In [13]:
sub_edges = []
for i, j, k in G.edges:
    if k.split(':')[0] in ['Monday', 'Tuesday', 'Wednesday']:
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.out_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[366682.6977999944, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [299994.94640000205, 'Grand Central-42 St (S,4,5,6,7)'],
 [198093.27320000465, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [178115.1026, '14 St-Union Sq (L,N,Q,R,W,4,5,6)'],
 [158768.4293999991, 'Fulton St (A,C,J,Z,2,3,4,5)']]

### 1C

In [14]:
sub_edges = []
for i, j, k in G.edges:
    if k.split(':')[0] in ['Saturday', 'Sunday']:
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.out_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[151484.71489999883, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [81496.38929999997, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [78418.81549999984, 'Grand Central-42 St (S,4,5,6,7)'],
 [77392.21220000015, '14 St-Union Sq (L,N,Q,R,W,4,5,6)'],
 [69408.95950000007, '34 St-Penn Station (A,C,E)']]

### 1D

In [15]:
sub_edges = []
for i, j, k in G.edges:
    if int(k.split(':')[1]) in list(range(1,5)):
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.out_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[11943.337700000026, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [6099.978999999999, '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)'],
 [6082.900499999989, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [5201.489999999985, 'W 4 St-Wash Sq (A,C,E,B,D,F,M)'],
 [4593.806199999965, '14 St-Union Sq (L,N,Q,R,W,4,5,6)']]

### 1E

In [16]:
sub_edges = []
for i, j, k in G.edges:
    if int(k.split(':')[1]) in list(range(6,9)):
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.out_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[131921.67709999942, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [89152.01950000029, 'Grand Central-42 St (S,4,5,6,7)'],
 [69550.92989999951, '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)'],
 [66670.79319999907, '34 St-Penn Station (1,2,3)'],
 [66484.19239999997, 'Flushing-Main St (7)']]

## 2

### 2A

In [17]:
# IN DEGREE FOR DESTINATION

boroughs = ['M', 'Q', 'Bk', 'Bx', 'SI']
borough_ridership = {}
for b in boroughs:
    filtered = nx.subgraph_view(G, filter_node=lambda x: G.nodes[x]["Borough"] == b)
    ridership = []
    for n in filtered.nodes:
        ridership.append([filtered.in_degree(n, weight='Estimated Average Ridership'), n])
    ridership = sorted(ridership, reverse=True)
    borough_ridership[b] = ridership[:5]
print(borough_ridership)

{'M': [[474085.9545000002, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'], [344473.24150000006, 'Grand Central-42 St (S,4,5,6,7)'], [224767.60169999904, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'], [214544.44960000043, '14 St-Union Sq (L,N,Q,R,W,4,5,6)'], [202620.20410000064, '59 St-Columbus Circle (A,B,C,D,1)']], 'Q': [[108329.4059999994, '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)'], [107684.79850000014, 'Flushing-Main St (7)'], [48124.096399999864, '103 St-Corona Plaza (7)'], [45009.05699999995, 'Junction Blvd (7)'], [42103.07600000032, 'Forest Hills-71 Av (E,F,M,R)']], 'Bk': [[103014.17410000041, 'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)'], [61373.84729999979, 'Jay St-MetroTech (A,C,F,R)'], [49045.45229999962, 'Court St (R)/Borough Hall (2,3,4,5)'], [41980.95460000002, 'Bedford Av (L)'], [38973.033599999726, 'Kings Hwy (B,Q)']], 'Bx': [[20726.217900000007, '161 St-Yankee Stadium (B,D,4)'], [18685.011899999943, '3 Av-149 St (2,5)'], [13192.732900000003, 'Parkchester (6)'],

### 2B

In [18]:
sub_edges = []
for i, j, k in G.edges:
    if k.split(':')[0] in ['Thursday', 'Friday']:
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.in_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[254027.95149999918, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [193929.87070000198, 'Grand Central-42 St (S,4,5,6,7)'],
 [146122.45519999982, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [137625.2514999999, '14 St-Union Sq (L,N,Q,R,W,4,5,6)'],
 [112550.08569999953, 'Fulton St (A,C,J,Z,2,3,4,5)']]

### 2C

In [19]:
sub_edges = []
for i, j, k in G.edges:
    if k.split(':')[0] in ['Saturday']:
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.in_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[82215.3906, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [48560.38809999999, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [48271.31670000009, '14 St-Union Sq (L,N,Q,R,W,4,5,6)'],
 [40886.46990000038, 'Grand Central-42 St (S,4,5,6,7)'],
 [35605.25350000015, '34 St-Penn Station (A,C,E)']]

### 2D

In [20]:
sub_edges = []
for i, j, k in G.edges:
    if int(k.split(':')[1]) in list(range(0,5)):
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.in_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[13370.50710000007, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [9410.559700000003, '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)'],
 [8158.897899999926, 'Grand Central-42 St (S,4,5,6,7)'],
 [6672.982799999966, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [5686.025199999945, '34 St-Penn Station (A,C,E)']]

### 2E

In [21]:
sub_edges = []
for i, j, k in G.edges:
    if int(k.split(':')[1]) in list(range(18,21)):
        sub_edges.append((i, j ,k))
filtered = G.edge_subgraph(sub_edges)
ridership = []
for n in filtered.nodes:
    ridership.append([filtered.in_degree(n, weight='Estimated Average Ridership'), n])
ridership = sorted(ridership, reverse=True)
top_5 = ridership[:5]
top_5

[[113725.83630000001, 'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)'],
 [57615.0811999999, 'Grand Central-42 St (S,4,5,6,7)'],
 [55440.312299999976, '34 St-Herald Sq (B,D,F,M,N,Q,R,W)'],
 [52727.58670000014, '14 St-Union Sq (L,N,Q,R,W,4,5,6)'],
 [51699.940900000096, '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)']]

## 3

### 3A

In [58]:
sub_edges = []
g2 = nx.DiGraph()
for i, j, k in G.edges:
    # check day / time
    if int(k.split(':')[1]) == 13 and k.split(':')[0] == "Monday":
        if not g2.has_edge(i, j):
            g2.add_edge(i, j, weight=G[i][j][k]['Estimated Average Ridership'])
        else:
            g2[i][j]['weight'] += G[i][j][k]['Estimated Average Ridership']

ridership = sorted(g2.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)
top_10 = ridership[:10]
top_10

[('Grand Central-42 St (S,4,5,6,7)',
  '14 St-Union Sq (L,N,Q,R,W,4,5,6)',
  {'weight': 134.784}),
 ('Fulton St (A,C,J,Z,2,3,4,5)',
  'Grand Central-42 St (S,4,5,6,7)',
  {'weight': 125.6758}),
 ('Flushing-Main St (7)', 'Junction Blvd (7)', {'weight': 123.3308}),
 ('Flushing-Main St (7)',
  '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)',
  {'weight': 122.4416}),
 ('Junction Blvd (7)', 'Flushing-Main St (7)', {'weight': 120.5736}),
 ('Grand Central-42 St (S,4,5,6,7)',
  'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)',
  {'weight': 119.5378}),
 ('Grand Central-42 St (S,4,5,6,7)',
  'Fulton St (A,C,J,Z,2,3,4,5)',
  {'weight': 115.57980000000002}),
 ('Grand Central-42 St (S,4,5,6,7)', '86 St (4,5,6)', {'weight': 114.6836}),
 ('14 St-Union Sq (L,N,Q,R,W,4,5,6)',
  'Times Sq-42 St (N,Q,R,W,S,1,2,3,7)/42 St (A,C,E)',
  {'weight': 113.7334}),
 ('14 St-Union Sq (L,N,Q,R,W,4,5,6)',
  'Grand Central-42 St (S,4,5,6,7)',
  {'weight': 112.1886})]

### 3B

In [55]:
sub_edges = []
g2 = nx.DiGraph()
for i, j, k in G.edges:
    source_borough = G.nodes[i]['Borough']
    destination_borough = G.nodes[j]['Borough']
    # check boroughs
    if source_borough == 'Q' and destination_borough == 'Q':
        # check day / time
        if int(k.split(':')[1]) in list(range(18,21)) and k.split(':')[0] == "Friday":
            if not g2.has_edge(i, j):
                g2.add_edge(i, j, weight=G[i][j][k]['Estimated Average Ridership'])
            else:
                g2[i][j]['weight'] += G[i][j][k]['Estimated Average Ridership']

ridership = sorted(g2.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)
top_10 = ridership[:10]
top_10

[('Flushing-Main St (7)',
  '74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)',
  {'weight': 526.1126}),
 ('Flushing-Main St (7)', '103 St-Corona Plaza (7)', {'weight': 513.9525}),
 ('Flushing-Main St (7)', 'Junction Blvd (7)', {'weight': 476.8428}),
 ('Junction Blvd (7)', 'Flushing-Main St (7)', {'weight': 385.0942}),
 ('Flushing-Main St (7)',
  '90 St-Elmhurst Av (7)',
  {'weight': 311.59180000000003}),
 ('74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)',
  'Flushing-Main St (7)',
  {'weight': 281.7269}),
 ('103 St-Corona Plaza (7)', 'Flushing-Main St (7)', {'weight': 242.8945}),
 ('82 St-Jackson Hts (7)',
  'Flushing-Main St (7)',
  {'weight': 207.61800000000002}),
 ('90 St-Elmhurst Av (7)',
  'Flushing-Main St (7)',
  {'weight': 181.40320000000003}),
 ('74-Broadway (7)/Jackson Hts-Roosevelt Av (E,F,M,R)',
  'Jamaica-179 St (F)',
  {'weight': 178.9214})]

### 3C

In [53]:
sub_edges = []
g2 = nx.DiGraph()
for i, j, k in G.edges:
    source_borough = G.nodes[i]['Borough']
    destination_borough = G.nodes[j]['Borough']
    # check boroughs
    if source_borough == 'Bk' and destination_borough == 'Bk':
        # check day / time
        if int(k.split(':')[1]) in list(range(1,5)):
            if not g2.has_edge(i, j):
                g2.add_edge(i, j, weight=G[i][j][k]['Estimated Average Ridership'])
            else:
                g2[i][j]['weight'] += G[i][j][k]['Estimated Average Ridership']

ridership = sorted(g2.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)
top_10 = ridership[:10]
top_10

[('Bedford Av (L)',
  'Myrtle-Wyckoff Avs (L,M)',
  {'weight': 102.15010000000002}),
 ('Bedford Av (L)', 'DeKalb Av (L)', {'weight': 80.53}),
 ('Lorimer St (L)/Metropolitan Av (G)',
  'Myrtle-Wyckoff Avs (L,M)',
  {'weight': 73.5999}),
 ('Jefferson St (L)',
  'Myrtle-Wyckoff Avs (L,M)',
  {'weight': 69.86269999999999}),
 ('Lorimer St (L)/Metropolitan Av (G)',
  'DeKalb Av (L)',
  {'weight': 59.859300000000005}),
 ('Jefferson St (L)', 'Bedford Av (L)', {'weight': 58.54900000000002}),
 ('Bedford Av (L)', 'Jefferson St (L)', {'weight': 56.6384}),
 ('Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)',
  '36 St (D,N,R)',
  {'weight': 54.46099999999999}),
 ('Flatbush Av-Brooklyn College (2,5)',
  'Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)',
  {'weight': 50.424}),
 ('Bedford Av (L)', 'Morgan Av (L)', {'weight': 50.134899999999995})]

### 3D

In [50]:
sub_edges = []
g2 = nx.DiGraph()
for i, j, k in G.edges:
    source_borough = G.nodes[i]['Borough']
    destination_borough = G.nodes[j]['Borough']
    # check boroughs
    if source_borough == 'Bk' and destination_borough == 'M':
        # check day / time
        if int(k.split(':')[1]) in list(range(6,7)) and k.split(':')[0] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday']:
            if not g2.has_edge(i, j):
                g2.add_edge(i, j, weight=G[i][j][k]['Estimated Average Ridership'])
            else:
                g2[i][j]['weight'] += G[i][j][k]['Estimated Average Ridership']

ridership = sorted(g2.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)
top_10 = ridership[:10]
top_10

[('Atlantic Av-Barclays Ctr (B,D,N,Q,R,2,3,4,5)',
  'Bowling Green (4,5)',
  {'weight': 326.71340000000004}),
 ('Crown Hts-Utica Av (3,4)',
  'Grand Central-42 St (S,4,5,6,7)',
  {'weight': 240.9032}),
 ('Flatbush Av-Brooklyn College (2,5)',
  'Grand Central-42 St (S,4,5,6,7)',
  {'weight': 208.6848}),
 ('Kings Hwy (B,Q)',
  '34 St-Herald Sq (B,D,F,M,N,Q,R,W)',
  {'weight': 200.7939}),
 ('Crown Hts-Utica Av (3,4)',
  'Fulton St (A,C,J,Z,2,3,4,5)',
  {'weight': 193.90349999999998}),
 ('Flatbush Av-Brooklyn College (2,5)',
  'Fulton St (A,C,J,Z,2,3,4,5)',
  {'weight': 186.75279999999998}),
 ('Crown Hts-Utica Av (3,4)', '86 St (4,5,6)', {'weight': 149.64079999999998}),
 ('Kings Hwy (B,Q)',
  '47-50 Sts-Rockefeller Ctr (B,D,F,M)',
  {'weight': 144.9144}),
 ('Kings Hwy (B,Q)', '72 St (Q)', {'weight': 142.09779999999998}),
 ('Crown Hts-Utica Av (3,4)',
  '14 St-Union Sq (L,N,Q,R,W,4,5,6)',
  {'weight': 142.0415})]

### 3E

In [47]:
sub_edges = []
g2 = nx.DiGraph()
for i, j, k in G.edges:
    source_borough = G.nodes[i]['Borough']
    destination_borough = G.nodes[j]['Borough']
    # check boroughs
    if source_borough == 'Bx' and destination_borough == 'M':
        # check day / time
        if int(k.split(':')[1]) in list(range(6,7)) and k.split(':')[0] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday']:
            if not g2.has_edge(i, j):
                g2.add_edge(i, j, weight=G[i][j][k]['Estimated Average Ridership'])
            else:
                g2[i][j]['weight'] += G[i][j][k]['Estimated Average Ridership']

ridership = sorted(g2.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)
top_10 = ridership[:10]
top_10

[('Parkchester (6)', 'Grand Central-42 St (S,4,5,6,7)', {'weight': 207.1318}),
 ('Parkchester (6)', '125 St (4,5,6)', {'weight': 186.91930000000002}),
 ('Parkchester (6)', '68 St-Hunter College (6)', {'weight': 174.5169}),
 ('Parkchester (6)', '86 St (4,5,6)', {'weight': 157.4212}),
 ('Parkchester (6)', 'Fulton St (A,C,J,Z,2,3,4,5)', {'weight': 145.2448}),
 ('Parkchester (6)',
  'Lexington Av-53 St (E,M)/51 St (6)',
  {'weight': 144.4392}),
 ('Woodlawn (4)', '86 St (4,5,6)', {'weight': 137.5386}),
 ('3 Av-149 St (2,5)',
  'Grand Central-42 St (S,4,5,6,7)',
  {'weight': 132.5911}),
 ('Parkchester (6)',
  'Brooklyn Bridge-City Hall (4,5,6)/Chambers St (J,Z)',
  {'weight': 126.886}),
 ('161 St-Yankee Stadium (B,D,4)',
  '34 St-Herald Sq (B,D,F,M,N,Q,R,W)',
  {'weight': 119.8167})]

### 3F

In [52]:
sub_edges = []
g2 = nx.DiGraph()
for i, j, k in G.edges:
    source_borough = G.nodes[i]['Borough']
    destination_borough = G.nodes[j]['Borough']
    # check boroughs
    if source_borough == 'SI' and destination_borough == 'M':
        # check day / time
        if int(k.split(':')[1]) in list(range(6,7)) and k.split(':')[0] in ['Monday', 'Tuesday', 'Wednesday', 'Thursday']:
            if not g2.has_edge(i, j):
                g2.add_edge(i, j, weight=G[i][j][k]['Estimated Average Ridership'])
            else:
                g2[i][j]['weight'] += G[i][j][k]['Estimated Average Ridership']

ridership = sorted(g2.edges(data=True),key= lambda x: x[2]['weight'],reverse=True)
top_10 = ridership[:10]
top_10

[]